In [ ]:
import pandas as pd
import re
import os

# --- CONFIGURACIÓN DE ARCHIVOS ---
EXCEL_FILE = "SMEAC_Master_Data.xlsx"
HTML_FILE = "index.html"
VERSION_HOJA = "v1.2"

# --- MAPEO DE TÍTULOS PROFESIONALES ---
PHASE_MAPPING = {
    "Phase 1": "Phase 1: Foundational Technology Optimization (Q3 - Q4 2025)",
    "Phase 2": "Phase 2: Centralization, Visibility & Tax Season Support (Q1 - Q2 2026)",
    "Phase 3": "Phase 3: Evolution & Growth (Q3 2026+)"
}

# ==============================================================================
# SECCIONES ESTRATÉGICAS SMEAC (Mantenidas según formato original)
# ==============================================================================

SITUATION_TEXT = """
<h2 class="font-title text-3xl font-bold text-gray-800 mb-6">Situation</h2>
<div class="space-y-4 text-gray-700">
    <p>J&W Accounting currently operates with significant departmental silos. Each department functions independently, with minimal coordination or integration.</p>
    <p>While we have invested substantially in modern software—most notably <strong>Karbon</strong> as our primary CRM—it has not been implemented or adopted effectively.</p>
</div>
"""

MISSION_TEXT = """
<h2 class="font-title text-3xl font-bold text-gray-800 mb-6">Mission</h2>
<div class="space-y-4 text-gray-700">
    <p>Over the course of <strong>12 months</strong>, the CTO will audit processes, optimize usage of software, and establish standardized technology-enabled processes.</p>
</div>
"""

ADMIN_TEXT = """
<h2 class="font-title text-3xl font-bold text-gray-800 mb-6">Administration & Logistics</h2>
<ul class="list-disc list-inside space-y-2 text-gray-700">
    <li><strong>Personnel:</strong> CTO (Miguel Vargas), Office Manager, Department Heads.</li>
    <li><strong>Tools:</strong> Karbon, Proseries, Square, etc.</li>
</ul>
"""

COMM_TEXT = """
<h2 class="font-title text-3xl font-bold text-gray-800 mb-6">Communication</h2>
<ul class="list-disc list-inside space-y-2 text-gray-700">
    <li><strong>Project Lead:</strong> CTO - Miguel Vargas.</li>
    <li><strong>Cadence:</strong> Weekly progress meetings, Monthly updates.</li>
</ul>
"""

# ==============================================================================
# LÓGICA DE PROCESAMIENTO ACTUALIZADA
# ==============================================================================

def format_date_clean(val):
    if pd.isna(val): return ""
    if hasattr(val, 'strftime'):
        try: return val.strftime('%b %d')
        except: return str(val)
    return str(val).strip().replace(' 00:00:00', '')

def crear_bloque_ejecucion_html(df, version):
    """ Genera el HTML con lógica de columna Key Milestone y validación de links """
    html_output = f"<h2 class='font-title text-3xl font-bold text-gray-800 mb-8 uppercase tracking-tight'>Execution Log - {version}</h2>"
    
    fases_presentes = ["Phase 1", "Phase 2", "Phase 3"]
    
    for fase in fases_presentes:
        if fase not in df['Phase'].values: continue
        
        fase_df = df[df['Phase'] == fase]
        full_title = PHASE_MAPPING.get(fase, fase)
        
        # Estilos visuales por fase
        if "1" in fase:
            bg_card, text_fase = "bg-blue-50 border-blue-200", "text-blue-900"
        elif "2" in fase:
            bg_card, text_fase = "bg-emerald-50 border-emerald-200", "text-emerald-900"
        else:
            bg_card, text_fase = "bg-slate-50 border-slate-200", "text-slate-800"
        
        html_output += f"""
        <div class="border rounded-xl overflow-hidden mb-12 shadow-sm {bg_card}">
            <div class="p-5 border-b flex justify-between items-center bg-white/50">
                <h3 class="font-title text-xl font-bold {text_fase} uppercase tracking-tight">{full_title}</h3>
                <span class="text-[10px] font-bold opacity-60 uppercase">{len(fase_df)} Activities logged</span>
            </div>
            <div class="p-6 bg-white/80">
        """
        
        secciones = fase_df['Section (Title)'].unique()
        for seccion in secciones:
            seccion_df = fase_df[fase_df['Section (Title)'] == seccion]
            
            # --- NUEVA LÓGICA DE LINK ---
            # Filtramos valores nulos, vacíos o "N/A"
            links_validos = seccion_df['Section Link'].dropna().astype(str).str.strip()
            links_validos = links_validos[links_validos.str.lower().replace('[|]', '', regex=True) != "n/a"]
            links_validos = links_validos[links_validos != ""]
            
            link_carpeta = links_validos.iloc[0] if not links_validos.empty else None
            
            # --- NUEVA LÓGICA DE KEY MILESTONE ---
            # Ahora lee directamente de la columna 'Key Milestone'
            has_milestone_col = 'Key Milestone' in seccion_df.columns
            is_key_milestone = False
            if has_milestone_col:
                is_key_milestone = any(seccion_df['Key Milestone'].astype(str).str.upper() == 'YES')
            
            border_class = "border-amber-400 border-2" if is_key_milestone else "border-gray-200"
            badge_milestone = '<span class="ml-4 bg-amber-100 text-amber-800 text-[9px] font-black px-2 py-0.5 rounded-sm border border-amber-200 uppercase tracking-widest">Key Milestone</span>' if is_key_milestone else ""

            # El botón solo se genera si el link no es None ni vacío
            btn_link = ""
            if link_carpeta and str(link_carpeta).lower() not in ["nan", "n/a", "none"]:
                # Limpiar posibles corchetes del formato markdown [Link]
                clean_url = re.sub(r'[\[\]]', '', str(link_carpeta))
                btn_link = f'<a href="{clean_url}" target="_blank" class="text-[10px] bg-blue-600 text-white py-1.5 px-3 rounded hover:bg-blue-700 transition-all ml-4 normal-case font-bold shadow-sm inline-flex items-center">View Docs <span class="ml-1">→</span></a>'

            html_output += f"""
            <div class="mb-12 last:mb-0 p-4 rounded-lg bg-white {border_class} shadow-sm relative">
                <div class="flex items-center mb-6 border-b border-gray-100 pb-2">
                    <h4 class="text-xs font-bold text-gray-600 uppercase tracking-widest">{seccion}</h4>
                    {badge_milestone} {btn_link}
                </div>
                <div class="space-y-8">
            """
            
            for _, row in seccion_df.iterrows():
                display_date = format_date_clean(row['Date'])
                impact_text = str(row['Impacto Estimado']) if 'Impacto Estimado' in row and pd.notna(row['Impacto Estimado']) else ""
                
                status_raw = str(row['Status']).lower()
                status_map = {"done": "status-done", "progress": "status-progress", "pending": "status-pending", "external": "status-external", "planning": "status-info"}
                status_class = status_map.get(status_raw, "status-info")
                
                impact_badge = f'<span class="bg-slate-100 text-slate-600 text-[10px] px-2 py-0.5 rounded border border-slate-200 ml-2 font-bold uppercase tracking-tighter">{impact_text}</span>' if impact_text else ""
                
                html_output += f"""
                <div class="flex items-start group">
                    <div class="w-24 flex-shrink-0 pt-1">
                        <span class="text-xs font-bold text-gray-800 font-mono tracking-tighter">{display_date}</span>
                    </div>
                    <div class="flex-grow pb-6 border-b border-gray-100 group-last:border-0 pl-6 border-l-2 border-gray-50 group-hover:border-blue-300 transition-colors">
                        <div class="flex justify-between items-start mb-1">
                            <div class="flex items-center flex-wrap gap-1">
                                <p class="text-sm font-bold text-gray-900 leading-snug">{row['Bitácora Entry (Literal Description)']}</p>
                                {impact_badge}
                            </div>
                            <span class="status {status_class} text-[11px] py-1 px-3 ml-4 flex-shrink-0 font-black uppercase shadow-sm">{row['Status']}</span>
                        </div>
                        <p class="text-xs text-gray-500 italic leading-relaxed mt-1">{row['Technical Detail / Business Impact']}</p>
                    </div>
                </div>
                """
            html_output += "</div></div>"
        html_output += "</div></div>"
    return html_output

# --- LÓGICA DE INYECCIÓN EN ARCHIVO HTML ---
def inyectar_seccion(html_content, marker_prefix, version, content):
    marker_start = f"<!-- {marker_prefix}_{version.upper()}_START -->"
    marker_end = f"<!-- {marker_prefix}_{version.upper()}_END -->"
    pattern = re.escape(marker_start) + ".*?" + re.escape(marker_end)
    replacement = f"{marker_start}\n{content}\n{marker_end}"
    return re.sub(pattern, replacement, html_content, flags=re.DOTALL)

def actualizar_dashboard_completo(html_path, exec_html, version):
    if not os.path.exists(html_path): 
        print(f"❌ Error: No se encontró el archivo base {html_path}")
        return
        
    with open(html_path, 'r', encoding='utf-8') as f: 
        contenido = f.read()

    contenido = inyectar_seccion(contenido, "PYTHON_SITUATION", version, SITUATION_TEXT)
    contenido = inyectar_seccion(contenido, "PYTHON_MISSION", version, MISSION_TEXT)
    contenido = inyectar_seccion(contenido, "PYTHON_EXECUTION", version, exec_html)
    contenido = inyectar_seccion(contenido, "PYTHON_ADMIN", version, ADMIN_TEXT)
    contenido = inyectar_seccion(contenido, "PYTHON_COMMUNICATION", version, COMM_TEXT)

    with open(html_path, 'w', encoding='utf-8') as f: 
        f.write(contenido)
    print(f"✅ Dashboard SMEAC v{version} actualizado con Key Milestones y validación de links.")

if __name__ == "__main__":
    if os.path.exists(EXCEL_FILE):
        try:
            df = pd.read_excel(EXCEL_FILE, sheet_name=VERSION_HOJA)
            html_gen = crear_bloque_ejecucion_html(df, VERSION_HOJA)
            actualizar_dashboard_completo(HTML_FILE, html_gen, VERSION_HOJA)
        except Exception as e:
            print(f"❌ Error procesando Excel: {e}")
    else:
        print(f"❌ No se encontró el archivo {EXCEL_FILE}")

✅ Dashboard SMEAC vv1.2 actualizado correctamente.
